In [ ]:
# Clone Repository of InstantCity
!rm -rf ./InstantCity/
!git clone https://github.com/ualsg/InstantCity

In [ ]:
# Change working directory to InstantCity
%cd InstantCity

In [ ]:
# What's inside
!ls

In [ ]:
!cat ./README.md

In [ ]:
!python --version

In [ ]:
# !pip install condacolab
# import condacolab
# condacolab.install()
# !conda --version
# !which conda

In [ ]:
# install modules
!pip install pathlib
!pip install dominate
!pip install scipy
!pip install torch
!pip install Pillow
!pip install torchvision

In [ ]:
# Connect to Google Drive to retreive GANmapper_Data
# from google.colab import drive
# drive.mount('/content/drive')
# !ls /content/drive/MyDrive/GANmapper_Data/

In [ ]:
# Copy the testing data from GANmapper_Data
# !cp -r /content/drive/MyDrive/GANmapper_Data/checkpoints /content/InstantCity/
# !cp -r /content/drive/MyDrive/GANmapper_Data/datasets /content/InstantCity/

In [ ]:
# Unzip GANmapper data
!unzip -q -o ../GANmapper\ Data.zip
# And move it to the current folders
!cp -r ./GANmapper\ Data/checkpoints .
!cp -r ./GANmapper\ Data/datasets .

In [ ]:
# What's inside datasets?
!ls ./datasets
!ls ./datasets/Exp4/

In [ ]:
# What's inside checkpoints?
!ls ./checkpoints/
!ls ./checkpoints/Exp3/
!ls ./checkpoints/Exp3/Singapore/

In [ ]:
# Copy modified python-scripts, to fix mistakes in the repo
!cp ../modified_files/train.py ./
!cp ../modified_files/test.py ./
!cp ../modified_files/pix2pixHD_model.py ./models/
!cp ../modified_files/models.py ./models/
!cp ../modified_files/image_folder.py ./data/
!cp ../modified_files/aligned_dataset.py ./data/

In [ ]:
# What's in the test script?
!cat train.py

In [ ]:
# What are the test's parameters?
!cat ./options/train_options.py

In [ ]:
# Create the training folder structure
!rm -r ./training_data
!mkdir ./training_data
!mkdir ./training_data/train_A
!mkdir ./training_data/train_B

In [ ]:
# Copy training images of Singapore to the training directory
!ls ./datasets/Exp4/
!cp -r ./datasets/Exp4/Singapore/Source/* ./training_data/train_A/
!cp -r ./datasets/Exp4/Singapore/Target/* ./training_data/train_B/

In [ ]:
# Copy training images of Allermöhe to the training directory
!cp -r ../data_training/allermoehe_source/* ./training_data/train_A/
!cp -r ../data_training/allermoehe_target/* ./training_data/train_B/

In [ ]:
# Copy training images of Allermöhe to the training directory
!cp -r ../data_training/allermoehe_source_2/* ./training_data/train_A/
!cp -r ../data_training/allermoehe_target_2/* ./training_data/train_B/

In [ ]:
# Run the training of a model
!python train.py --name custom_model --dataroot ./training_data --debug
# Remember to copy the model after training to a permanent location, that won't get deleted (oudside of InstantCity/InstantCity/)

# Copy Test-model
!mkdir ./checkpoints/SG15
print("Copying data...")
!cp -r ./checkpoints/Exp3/Singapore/* ./checkpoints/SG15/
print("Copy complete.")
!ls ./checkpoints/SG15/

In [ ]:
# Copy Test-dataset
!mkdir ./test
!rm -r ./test/Test
!mkdir ./test/Test
!ls ./datasets/Exp4/
print("Copying data...")
!cp -r ./datasets/Exp4/Paris/Source/* ./test/Test
print("Copy complete.")
!ls ./test/Test/

In [ ]:
# Copy Oberbillwerder-dataset
!mkdir ./test
!rm -r ./test/oberbillwerder
!mkdir ./test/oberbillwerder
!ls ../data_training/oberbillwerder_source
print("Copying data...")
!cp -r ../data_training/oberbillwerder_source/* ./test/oberbillwerder
print("Copy complete.")
!ls ./test/oberbillwerder/

In [ ]:
# Available models
!ls ./checkpoints/

In [ ]:
# Available test-datasets
!ls ./test/

In [ ]:
# Testing of the model
# --------------------

# Remember:
# The model must be available at "./checkpoints/{name}/"
# The the test images must be available at "./test/{dataroot}"
# !python test.py --name {name} --dataroot ./test/{dataroot}

!rm -r ./fake\\16/
!python test.py --name custom_model --dataroot ./test/oberbillwerder
!ls ./fake\\16/

In [ ]:
!ls ./fake\\16/
!ls ./fake\\16/33198/
# from PIL import Image
# im = Image.open("./fake\\16/33198/22542.png")
# im.show()